# Import Libraries

In [2]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
random.seed(530)
import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from scipy.io import loadmat
from tqdm.auto import tqdm
from glob import glob

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.layers import (concatenate, Conv2D, Dense, Dropout, BatchNormalization, MaxPooling2D,
                          Embedding, Flatten, Input, MaxPool2D, Activation, GlobalAveragePooling2D,
                          Reshape, GlobalAveragePooling2D, Layer)

# Data Load

In [3]:
# Set Data Path
data_path = "../MPIIGaze/Data/Normalized/" # 전처리가 수행된 데이터셋 경로 정의
mat_path = sorted(glob(os.path.join(data_path,"p*","day*.mat"))) # p1*으로 수집한 데이터 경로 정의
print(f"Find {len(mat_path)} Data") # 전체 mat 파일 수 출력

Find 521 Data


## mat 데이터를 데이터프레임으로 전체 변환

In [4]:
eye_loc_list = ["right", "left"] # 눈 위치 정의
# mat 파일 1개씩 작업 수행
datas = []
for idx in tqdm(range(len(mat_path))):
    cur_mat_path = mat_path[idx].replace("\\","/")
    par_id = cur_mat_path.split("/")[-2] # 참가자 id
    day_id = cur_mat_path.split("/")[-1].split(".")[0] # 수집 일자
    
    # load mat
    mat_data = loadmat(cur_mat_path, squeeze_me=True, struct_as_record=True)['data']
    # 눈 위치별 데이터 추출
    for loc in eye_loc_list:
        eye_loc_data = mat_data[loc].tolist()
        
        # 이미지 데이터
        image_data = eye_loc_data["image"].tolist()
        if len(image_data.shape) < 3:
            image_data = image_data[np.newaxis,:]
        
        # Head Pose 데이터
        pose_data = eye_loc_data["pose"].tolist()
        if len(pose_data.shape) < 2:
            pose_data = pose_data[np.newaxis,:]
        
        # Gaze 데이터
        gaze_data = eye_loc_data["gaze"].tolist()
        if len(gaze_data.shape) < 2:
            gaze_data = gaze_data[np.newaxis,:]
        
        for i in range(len(image_data)):
            pose = pose_data[i].tolist()
            gaze = gaze_data[i].tolist()
            
            data_list = [par_id, day_id, loc, image_data[i], pose[0], pose[1], pose[2], gaze[0], gaze[1], gaze[2]]
            datas.append(data_list)

# 리스트에 담아두었던 정보들을 DataFrame으로 생성
data_df = pd.DataFrame(columns=["participant_id","day","eye_location","image","pose_x","pose_y","pose_z","gaze_x","gaze_y","gaze_z"], data=datas)
data_df.head(3)

  0%|          | 0/521 [00:00<?, ?it/s]

,participant_id,day,eye_location,image,pose_x,pose_y,pose_z,gaze_x,gaze_y,gaze_z
0,p00,day01,right,"[[198, 210, 222, 233, 239, 243, 245, 246, 245,...",-0.151908,0.241725,0.018486,-0.122025,0.090110,-0.988428
1,p00,day01,right,"[[240, 245, 247, 246, 243, 238, 235, 233, 231,...",-0.175984,0.148409,0.013117,-0.099334,0.217299,-0.971038
2,p00,day01,right,"[[233, 241, 245, 244, 241, 238, 237, 233, 230,...",-0.186510,0.174603,0.016372,-0.154061,0.159956,-0.975028


## 오른쪽 눈 정보를 Y축으로 대칭

In [5]:
flipped_df = data_df.copy()
for i in tqdm(range(len(flipped_df))):
    eye_location = flipped_df["eye_location"].iloc[i]
    if eye_location == "right":
        # 이미지 좌우대칭
        org_image = flipped_df["image"].iloc[i]
        new_image  = np.flip(org_image, axis=1)
#         flipped_df.loc[i, "image"] = new_image
        flipped_df["image"].iloc[i] = new_image
        
        # 좌표 좌우대칭
        org_head_x = flipped_df["pose_x"].iloc[i]
        new_head_x = -org_head_x
        flipped_df["pose_x"].iloc[i] = new_head_x
#         flipped_df.loc[i, "pose_x"] = new_head_x
        
        org_gaze_x = flipped_df["gaze_x"].iloc[i]
        new_gaze_x = -org_gaze_x
        flipped_df["gaze_x"].iloc[i] = new_gaze_x
#         flipped_df.loc[i:, "gaze_x"] = new_gaze_x
flipped_df.head(3)

  0%|          | 0/427316 [00:00<?, ?it/s]

,participant_id,day,eye_location,image,pose_x,pose_y,pose_z,gaze_x,gaze_y,gaze_z
0,p00,day01,right,"[[110, 102, 96, 88, 84, 82, 82, 79, 79, 82, 82...",0.151908,0.241725,0.018486,0.122025,0.090110,-0.988428
1,p00,day01,right,"[[106, 100, 92, 90, 88, 88, 90, 90, 92, 92, 95...",0.175984,0.148409,0.013117,0.099334,0.217299,-0.971038
2,p00,day01,right,"[[96, 88, 85, 83, 88, 88, 88, 83, 80, 80, 83, ...",0.186510,0.174603,0.016372,0.154061,0.159956,-0.975028


## 각 참가자의 눈 위치별 1500개의 데이터 임의 추출

In [6]:
par_list = sorted(list(set(list(flipped_df['participant_id'].values))))

right_df = pd.DataFrame()
left_df = pd.DataFrame()
for i in range(len(par_list)):
    par_df = flipped_df[flipped_df["participant_id"] == par_list[i]]
    par_right_df = par_df[par_df["eye_location"] == "right"]
    par_left_df = par_df[par_df["eye_location"] == "left"]
    
    par_right_idx_list = list(par_right_df.index)
    par_left_idx_list = list(par_left_df.index)
    
    while True:
        if len(par_right_idx_list) < 1500:
            par_right_idx_list.extend(par_right_idx_list)  # 오타 수정
        else:
            break
    while True:
        if len(par_left_idx_list) < 1500:
            par_left_idx_list.extend(par_left_idx_list)  # 오타 수정
        else:
            break
    
    sample_right_idx_list = random.sample(par_right_idx_list, 1500)
    sample_left_idx_list = random.sample(par_left_idx_list, 1500)
    
    sample_right_df = par_right_df.loc[sample_right_idx_list]  # 수정: iloc 대신 loc를 사용해야 함
    sample_left_df = par_left_df.loc[sample_left_idx_list]  # 수정: iloc 대신 loc를 사용해야 함
    
    right_df = pd.concat([right_df, sample_right_df])
    left_df = pd.concat([left_df, sample_left_df])

menet_dataset = pd.concat([right_df, left_df])
menet_dataset = menet_dataset.sort_values(by=['participant_id', 'day']).reset_index(drop=True)

In [7]:
menet_dataset.tail(3)

,participant_id,day,eye_location,image,pose_x,pose_y,pose_z,gaze_x,gaze_y,gaze_z
44997,p14,day07,left,"[[121, 121, 121, 114, 114, 123, 106, 94, 110, ...",-0.050017,-0.060703,-0.001519,0.066417,0.059492,-0.996017
44998,p14,day07,left,"[[240, 239, 235, 231, 222, 210, 206, 210, 182,...",-0.111802,0.351175,0.019857,-0.249634,0.148032,-0.956958
44999,p14,day07,left,"[[124, 102, 112, 112, 96, 89, 102, 105, 96, 96...",-0.105359,0.188947,0.009993,0.109166,0.291560,-0.950303


# Set Parameters

In [8]:
# 전처리된 이미지의 크기 정보 정의
sample_img = menet_dataset['image'].iloc[0]
IMG_HEIGHT, IMG_WIDTH = sample_img.shape[0], sample_img.shape[1] # 36, 60

# 모델 학습이 필요한 파라미터 변수 정의
batch_size = 32
epochs = 100
patience = 15

# Target 변수 정의
head_col = ["pose_x", "pose_y", "pose_z"]
y_col = ["gaze_x","gaze_y","gaze_z"]

# 참가자 정보 중 몇명의 참가자가 있는지 count 수행한 변수 정의
n_cats = len(list(set(list(menet_dataset['participant_id'].values))))

# Get Data Generator

In [9]:
# Dataset Split 수행
def get_generators(df):
    # DataFrame에서 train/valid/test로 나누기 수행
    train, test = train_test_split(df, test_size=0.2, random_state=530)
    train, valid = train_test_split(train, test_size=0.2, random_state=530)

    # 각 데이터셋 별 인덱스 정보 추출
    train_idxs = train.index.to_numpy()
    valid_idxs = valid.index.to_numpy()
    test_idxs = test.index.to_numpy()

    # 각 데이터셋에는 이미지와 시선 벡터 정보만 X와 y로 구성되도록 작업 수행
    train_generator = ([np.stack(train["image"].to_list()), np.array(train[head_col].values)], np.array(train[y_col].values))
    valid_generator = ([np.stack(valid["image"].to_list()), np.array(valid[head_col].values)], np.array(valid[y_col].values))
    test_generator = ([np.stack(test["image"].to_list()), np.array(test[head_col].values)], np.array(test[y_col].values))

    return train_generator, valid_generator, test_generator, train_idxs, valid_idxs, test_idxs

train_generator, valid_generator, test_generator, train_idxs, valid_idxs, test_idxs = get_generators(menet_dataset)

# Define Menet Architecture

In [10]:
# 정의한 CNN 모델의 출력 전 Layer에서의 출력(feature map)을 가져오는 함수
def get_feature_map(model, X):
    last_layer = Model(inputs = model.input, outputs = model.layers[-5].output)
    return last_layer.predict(X, verbose=0)

In [11]:
# Negative Log Likelihood 계산 수식 함수
def nll_i(y_i, f_hat_i, Z_i, b_hat_i, R_hat_i, D_hat):
    total = 0
    for j in range(b_hat_i.shape[1]):
        b_col = b_hat_i[:, j]
        total += b_col.T @ np.linalg.inv(D_hat) @ b_col
    nll = (y_i - f_hat_i - Z_i @ b_hat_i).T @ np.linalg.inv(R_hat_i) @ (y_i - f_hat_i - Z_i @ b_hat_i) + total + (np.sum(np.log(np.diag(D_hat)))/D_hat.shape[0]) + (np.sum(np.log(np.diag(R_hat_i)))/R_hat_i.shape[0])

    return nll

In [12]:
# Early Stop 기능 함수
def check_stop_model(nll_valid, best_loss, wait_loss, patience):
    stop_model = False
    if nll_valid < best_loss:
        best_loss = nll_valid
        wait_loss = 0
    else:
        wait_loss += 1
        if wait_loss >= patience:
            stop_model = True
    return best_loss, wait_loss, stop_model

In [21]:
# Negative Log Likelihood 계산을 위한 함수
def compute_nll_generator(model, generator, b_hat, D_hat, sig2e_est, maps2ind, n_clusters, cnt_clusters):
    inputs_data, labels = generator
    inputs = inputs_data[0]
    head_pose = inputs_data[1]
    inputs = np.reshape(inputs, (inputs.shape[0], inputs.shape[1], inputs.shape[2], 1))
    y = np.array(labels)

    f_hat = model.predict([inputs, head_pose], verbose=0).reshape(inputs.shape[0], 3)
    Z = get_feature_map(model, [inputs, head_pose])
    nll = 0
    for cluster_id in range(n_clusters):
        indices_i = maps2ind[cluster_id]
        n_i = cnt_clusters[f"p{str(cluster_id).zfill(2)}"]
        y_i = y[indices_i]
        Z_i = Z[indices_i, :]
        f_hat_i = f_hat[indices_i]

        reg_factor = 1e-1
        I_i = np.eye(n_i) * reg_factor
        R_hat_i = sig2e_est * I_i

        b_hat_i = b_hat[cluster_id, :]
        nll_i_val = nll_i(y_i, f_hat_i, Z_i, b_hat_i, R_hat_i, D_hat)
        nll += nll_i_val
    return nll

In [14]:
# MeNet 학습 함수
def menet_fit_generator(model, train_generator, valid_generator, clusters_train, clusters_valid, n_clusters, epochs, callbacks, patience, verbose=1):
    train_data, y_train = train_generator # 학습 데이터 호출
    X_train = train_data[0]
    Head_pose = train_data[1]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)) # X 데이터 정의 및 이미지 형태로 reshape
    y_train = np.array(y_train) # y 데이터 정의

    # 각 참가자에 해당하는 데이터의 위치를 매핑하기 위한 정보 리스트
    maps2ind_train = [list(np.where(clusters_train == f"p{str(i).zfill(2)}")[0]) for i in range(n_clusters)]
    maps2ind_valid = [list(np.where(clusters_valid == f"p{str(i).zfill(2)}")[0]) for i in range(n_clusters)]

    # 각 참가자의 데이터가 어느 정도 있는지 count 하는 구문
    cnt_clusters_train = Counter(clusters_train)
    cnt_clusters_valid = Counter(clusters_valid)

    # 초기 값 정의
    Z = get_feature_map(model, [X_train, Head_pose])
    d = Z.shape[1]
    b_hat = np.zeros((n_clusters, d , y_train.shape[1]))
    D_hat = np.eye(d)
    sig2e_est = 1.0

    nll_history = {'train': [], 'valid': []}
    best_loss = np.inf
    wait_loss = 0

    # 학습 수행
    for epoch in range(epochs):
        ts = time.time() # 학습 시작 초
        print("E-Step")
        y_star = np.zeros(y_train.shape) # y_start를 y 데이터의 shape의 크기를 갖는 0의 배열로 정의
        # 각 참가자별 y_star 계산
        for cluster_id in range(n_clusters):
            indices_i = maps2ind_train[cluster_id]
            b_hat_i = b_hat[cluster_id, :]
            y_star_i = y_train[indices_i] - Z[indices_i, :] @ b_hat_i
            y_star[indices_i] = y_star_i
        # y_star 추론
        model.fit([X_train, Head_pose], y_star, epochs=1, verbose=1)

        # y_star로 학습한 모형에서 X에 대하여 feature map 추출
        Z = get_feature_map(model, [X_train, Head_pose])
        # f_hat 추론
        f_hat = model.predict([X_train, Head_pose], verbose=0).reshape(X_train.shape[0], 3)

        print("M-Step")
        sig2e_est_sum = 0
        D_hat_sum = 0
        for cluster_id in range(n_clusters):
            indices_i = maps2ind_train[cluster_id]
            n_i = cnt_clusters_train[f"p{str(cluster_id).zfill(2)}"]
            f_hat_i = f_hat[indices_i] # 해당 클러스터에 속하는 샘플의 예측값
            y_i = y_train[indices_i] # 해당 클러스터에 속하는 샘플의 실제값
            Z_i = Z[indices_i, :] # 해당 클러스터에 속하는 샘플의 feature map
            
            # V_hat 계산
            V_hat_i = Z_i @ D_hat @ np.transpose(Z_i) + sig2e_est * np.eye(n_i)
            V_hat_inv_i =  np.linalg.inv(V_hat_i) # V_hat의 역행렬
            
            # b_hat 갱신
            b_hat_i = D_hat @ np.transpose(Z_i) @ V_hat_inv_i @ (y_i - f_hat_i)
            eps_hat_i = y_i - f_hat_i - Z_i @ b_hat_i # 잔차들의 합
            b_hat[cluster_id, :] = b_hat_i
            
            # sig2e 추정값의 합 갱신
            residual_sum = np.sum(eps_hat_i**2)
            sig2e_est_sum = sig2e_est_sum + residual_sum + sig2e_est * (n_i - sig2e_est * np.trace(V_hat_inv_i))
            # D_hat의 합 갱신
            D_hat_sum = D_hat_sum + b_hat_i @ np.transpose(b_hat_i) + (D_hat - D_hat @ np.transpose(Z_i) @ V_hat_inv_i @ Z_i @ D_hat)
        sig2e_est = sig2e_est_sum / X_train.shape[0] # sig2e 추정값 갱신
        D_hat = D_hat_sum / n_clusters # D_hat 계산

        # nll 계산 수행 구문
        nll_train = compute_nll_generator(model, train_generator, b_hat, D_hat, sig2e_est, maps2ind_train, n_clusters, cnt_clusters_train)
        nll_train_sum = np.sum(nll_train)/y_train.shape[1]
        
        nll_valid = compute_nll_generator(model, valid_generator, b_hat, D_hat, sig2e_est, maps2ind_valid, n_clusters, cnt_clusters_valid)
        nll_valid_sum = np.sum(nll_valid)/y_train.shape[1]
        
        # nll 계산 결과 저장하는 리스트
        nll_history['train'].append(nll_train_sum)
        nll_history['valid'].append(nll_valid_sum)
        # 최적 모델을 위한 early stopping 구현
        best_loss, wait_loss, stop_model = check_stop_model(nll_valid_sum, best_loss, wait_loss, patience)
        te = time.time()
        
        if verbose:
            print(f'epoch: {epoch}, train_loss: {nll_train_sum:.2f}, val_loss: {nll_valid_sum:.2f}, sig2e_est: {sig2e_est:.2f}, time : {te-ts}\n')
        if stop_model:
            break
    n_epochs = len(nll_history['valid'])
    return model, b_hat, sig2e_est, n_epochs, nll_history

In [15]:
def menet_predict_generator(model, test_generator, clusters, n_clusters, b_hat):
    test_data, y_test = test_generator
    X_test = test_data[0]
    Head_pose = test_data[1]
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

    y_hat = model.predict([X_test, Head_pose], verbose=0).reshape(X_test.shape[0],3)
    Z = get_feature_map(model, [X_test, Head_pose])
    
    for cluster_id in range(n_clusters):
        indices_i = np.where(clusters == f"p{str(cluster_id).zfill(2)}")[0]
        if len(indices_i) == 0:
            continue
        b_i = b_hat[cluster_id, :]
        Z_i = Z[indices_i, :]
        y_hat[indices_i] = y_hat[indices_i] + Z_i @ b_i
    return y_hat

# Define CNN Model

In [16]:
def cnn_ignore():
    input_layer = Input((IMG_HEIGHT, IMG_WIDTH, 1))
    x = Conv2D(32, (5, 5), activation='relu')(input_layer)
    x = Conv2D(64, (5, 5), activation='relu')(x)
    x = Conv2D(32, (5, 5), activation='relu')(x)
    x = Conv2D(16, (5, 5), activation='relu')(x)
    x = MaxPool2D((2, 2))(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(3)(x)
    return Model(inputs=[input_layer], outputs=output)

# Define ResNet-18
- MeNet 2019

In [17]:
def resnet_block(inputs, filters, kernel_size=3, stride=1, conv_shortcut=True):
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters, kernel_size=kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
    if conv_shortcut:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride)(inputs)
        shortcut = BatchNormalization()(shortcut)
        x = tf.keras.layers.add([x, shortcut])
    else:
        x = tf.keras.layers.add([x, inputs])
    
    x = Activation('relu')(x)
    return x

def build_resnet18():
    img_inputs = Input((IMG_HEIGHT, IMG_WIDTH, 1))
    pose_inputs = Input(3)
    
    # ResNet Architecture
    x = Conv2D(64, kernel_size=7, strides=2, padding='same')(img_inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    
    x = resnet_block(x, 64, conv_shortcut=False)
    x = resnet_block(x, 64)
    x = resnet_block(x, 64)
    
    x = resnet_block(x, 128, stride=2)
    x = resnet_block(x, 128)
    x = resnet_block(x, 128)
    
    x = resnet_block(x, 256, stride=2)
    x = resnet_block(x, 256)
    x = resnet_block(x, 256)
    
    x = resnet_block(x, 512, stride=2)
    x = resnet_block(x, 512)
    x = resnet_block(x, 512)
    
    x = GlobalAveragePooling2D()(x)
    
    # Head Pose와 결합하기 위하여 Conv 배열 Flatten Feature Map 생성
    x = Flatten()(x)
    # Image Feature Map과 Head Pose Concatenate
    x = concatenate([x, pose_inputs])
    # Model 출력
    outputs = Dense(3, activation='relu')(x)
    
    model = Model([img_inputs, pose_inputs], outputs)
    return model

# Model Training

In [22]:
model = build_resnet18() # cnn_ignore()
model.compile(loss='mse', optimizer='adam')
callbacks = [EarlyStopping(monitor='val_loss', patience=epochs if patience is None else patience)]

In [19]:
clusters_train = np.array([menet_dataset['participant_id'][idx] for idx in train_idxs])
clusters_valid = np.array([menet_dataset['participant_id'][idx] for idx in valid_idxs])
clusters_test = np.array([menet_dataset['participant_id'][idx] for idx in test_idxs])

In [23]:
train_time = time.time()
model, b_hat, sig2e_est, n_epochs, _ = menet_fit_generator(model, train_generator, valid_generator,
                                                           clusters_train, clusters_valid, n_cats,epochs=epochs,
                                                           callbacks=callbacks, patience=patience, verbose=1)

print("="*20)
print(f"Total Training Time : {time.time() - train_time}(s)")
print("="*20)

(28800, 3)
E-Step
900/900 [==============================] - 48s 50ms/step - loss: 0.3340
M-Step
epoch: 0, train_loss: 6640.95, val_loss: 2366.10, sig2e_est: 0.16, time : 95.6845030784607

E-Step
900/900 [==============================] - 46s 50ms/step - loss: 0.0034
M-Step
epoch: 1, train_loss: 24010.24, val_loss: 9787.92, sig2e_est: 0.04, time : 93.83336901664734

E-Step
873/900 [============================>.] - ETA: 1s - loss: 0.0030

In [ ]:
y_pred = menet_predict_generator(model, test_generator, clusters_test, n_cats, b_hat).reshape(len(test_generator[0]),3)

In [ ]:
pred_df = pd.DataFrame(columns=["x_pred","y_pred","z_pred"],data=y_pred)
pred_df.to_csv("./result/menet_result.csv",index=False)

In [ ]:
real_df = pd.DataFrame(columns=["x_real","y_real","z_real"], data=test_generator[1])
real_df.to_csv("./result/menet_real.csv",index=False)

# Evalutation
- 3d vector dot

In [ ]:
print("Y Predict Sample")
print(y_pred[:5])
print()
print("Y Real Sample")
print(test_generator[1][:5])

In [ ]:
dot_list = []
for i in range(len(y_pred)):
    y_hat = y_pred[i]
    y_trh = test_generator[1][i]
    
    vector_dot = np.dot(y_trh, y_hat)
    dot_list.append(vector_dot)

mean_dot = (sum(dot_list)/len(dot_list))
max_dot = max(dot_list)
min_dot = min(dot_list)

print("Mean : ", mean_dot)
print("Max : ", max_dot)
print("Min : ", min_dot)

In [ ]:
# cosin ceta

# 10월 중순 발표
# 19년도에 논문으로 구현(target 변수, resnet)
# NNNN코드에 대하여 gaze 데이터에 대하여 적용